In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')) # or the path to your source code
sys.path.insert(0, module_path)

In [3]:
import pandas as pd
from tqdm import tqdm
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from nlp_project.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR
from nlp_project.plots import *
from nlp_project.dataset import build_ontolgy

2024-10-24 11:32:46.321 | INFO     | nlp_project.config:<module>:11 - PROJ_ROOT path is: C:\Users\Utente\OneDrive - Università di Salerno\Magistrale\NLP\nlp-project


In [5]:
dataset = pd.read_csv(PROCESSED_DATA_DIR / 'preprocessed_dataset.csv', sep=';')
dataset

,overall,reviewText,description,title,brand,price,asin,year,preprocessed_text,tagged_text
0,5.0,"I just got mine today, but so far so good! I ...",['Like new in box great looking phone get it b...,Motorola Moto Q - Smartphone - CDMA2000 1X - b...,Motorola,17.99,B000FYU4SO,2006,got mine today far good used use t618 obviousl...,"[('got', 'VERB', 0.0), ('mine', 'ADJ', 0.0), (..."
1,1.0,I've spent a week now trying to get this to sy...,['Like new in box great looking phone get it b...,Motorola Moto Q - Smartphone - CDMA2000 1X - b...,Motorola,17.99,B000FYU4SO,2006,ive spent week trying get synch office compute...,"[('ive', 'ADJ', 0.0), ('spent', 'VERB', 0.0), ..."
2,5.0,I have read a lot of the reviews and spoken to...,['Like new in box great looking phone get it b...,Motorola Moto Q - Smartphone - CDMA2000 1X - b...,Motorola,17.99,B000FYU4SO,2006,read lot review spoken number current owner mo...,"[('read', 'VERB', 0.0), ('lot', 'NOUN', 0.0), ..."
3,2.0,I had been waiting for months for this phone t...,['Like new in box great looking phone get it b...,Motorola Moto Q - Smartphone - CDMA2000 1X - b...,Motorola,17.99,B000FYU4SO,2006,waiting month phone replace blackberry 7100 wa...,"[('waiting', 'VERB', 0.0), ('month', 'NOUN', 0..."
4,5.0,this is THE phone. its like the razor and the...,['Like new in box great looking phone get it b...,Motorola Moto Q - Smartphone - CDMA2000 1X - b...,Motorola,17.99,B000FYU4SO,2006,phone like razor blackberry baby blackberry ri...,"[('phone', 'NOUN', 0.0), ('like', 'ADP', 2.0),..."
...,...,...,...,...,...,...,...,...,...,...
106158,5.0,"My old 5C was dying, quickly. I was in a hurr...",['iPhone with iOS 9; Lightning to USB Cable; U...,"Apple iPhone SE, GSM Unlocked, 16 GB - Space G...",Apple,40.08,B01GXAT0BK,2018,old 5c dying quickly hurry get replacement kic...,"[('old', 'ADJ', 0.0), ('5c', 'NUM', 0.0), ('dy..."
106159,4.0,Overall good buy,['iPhone with iOS 9; Lightning to USB Cable; U...,"Apple iPhone SE, GSM Unlocked, 16 GB - Space G...",Apple,40.08,B01GXAT0BK,2018,overall good buy,"[('overall', 'ADJ', 0.0), ('good', 'ADJ', 3.0)..."
106160,1.0,This phone is no good!! Keeps freezing up. Eve...,['iPhone with iOS 9; Lightning to USB Cable; U...,"Apple iPhone SE, GSM Unlocked, 16 GB - Space G...",Apple,40.08,B01GXAT0BK,2018,phone good keep freezing even touching power h...,"[('phone', 'NOUN', 0.0), ('good', 'ADJ', 3.0),..."
106161,5.0,Broke my phone and needed an urgent substituti...,['iPhone with iOS 9; Lightning to USB Cable; U...,"Apple iPhone SE, GSM Unlocked, 16 GB - Space G...",Apple,40.08,B01GXAT0BK,2018,broke phone needed urgent substitution phone r...,"[('broke', 'VERB', -1.0), ('phone', 'NOUN', 0...."


In [ ]:
descr = dataset[['description', 'asin', 'brand']]
descr.drop_duplicates(inplace=True)
descr

In [ ]:
brands_product_count = descr.groupby('brand').size().reset_index(name='counts').sort_values(by='counts', ascending=False)
brands_product_count.to_csv(INTERIM_DATA_DIR / 'brand_product_count.csv',sep=';', index=False)
brands_product_count

In [ ]:
import plotly.graph_objects as go

labels = brands_product_count['brand']
values = brands_product_count['counts']

pie_chart_go(labels=labels, values=values)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
'''
def build_ontolgy(df):
    nouns = []
    for index, row in tqdm(df.iterrows(), total = len(df)):
        text = row['description']
        text_cleaned = re.sub(r'[^\w\s]', '', text) #cleaning
        words = nltk.word_tokenize(text_cleaned) #tokenization
        words = [w.lower() for w in words] #lower casing
        tagged_words = nltk.pos_tag(words, tagset='universal') #POS-tagging
        preprocessed_words = [(lemmatizer.lemmatize(w), tag) for w, tag in tagged_words if not w in stop_words] #stopwords removal and lemmatization
        #print(filter_text)
        for word, tag in preprocessed_words:
            if(tag == 'NOUN'):
                nouns.append(word)
            
    return nouns
'''

In [ ]:
ont = build_ontolgy(descr, lemmatizer, stop_words)

In [ ]:
from collections import Counter

nouns_freq = Counter(ont)
#print(value_counts)
# Convert Counter to DataFrame
df_nouns_freq = pd.DataFrame(nouns_freq.items(), columns=['value', 'count'])
df_nouns_freq = df_nouns_freq.sort_values(by='count', ascending=False)
df_nouns_freq.head(30)

In [ ]:
sb_bar_plot(x = df_nouns_freq['count'][:25],
            y = df_nouns_freq['value'][:25],
            orient='h',
            title='Most frequent nouns in description', 
            xlabel='Count', 
            ylabel='Noun')

Esporto e analizzo in excel manualmente

In [ ]:
df_nouns_freq.to_csv(PROCESSED_DATA_DIR / 'ontology.csv', index=False, sep=';')

In [ ]:
# Parole chiave da cercare
#features = list(pd.read_csv('data/ontology_filtered.csv', sep=';')['value'])


In [ ]:
#features

In [ ]:
# Crea un'espressione regolare che combini tutte le parole chiave
#pattern = '|'.join(features)

# Filtra il DataFrame
#filtered_df = dataset[dataset['preprocessedText'].str.contains(pattern, case=False, na=False)]

#filtered_df.loc[0,'preprocessedText']

In [ ]:
#filtered_df.to_csv('data/preprocessed_dataset.csv', index=False, sep=';')